In [84]:
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kseniapetuhova/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [85]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
import pyLDAvis
import pyLDAvis.gensim  
import matplotlib.pyplot as plt
%matplotlib inline
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [86]:
import collections

In [87]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

# Прогоним gensim (mallet не работает)

In [88]:
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
print(df.target_names.unique())
df.head()

['rec.autos' 'comp.sys.mac.hardware' 'comp.graphics' 'sci.space'
 'talk.politics.guns' 'sci.med' 'comp.sys.ibm.pc.hardware'
 'comp.os.ms-windows.misc' 'rec.motorcycles' 'talk.religion.misc'
 'misc.forsale' 'alt.atheism' 'sci.electronics' 'comp.windows.x'
 'rec.sport.hockey' 'rec.sport.baseball' 'soc.religion.christian'
 'talk.politics.mideast' 'talk.politics.misc' 'sci.crypt']


,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space


In [89]:
data = df.content.values.tolist()
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
data = [re.sub('\s+', ' ', sent) for sent in data]
data = [re.sub("\'", "", sent) for sent in data]

In [90]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
data_words = list(sent_to_words(data))

In [91]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [92]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [93]:
data_words_nostops = remove_stopwords(data_words)
data_words_bigrams = make_bigrams(data_words_nostops)
nlp = spacy.load('en', disable=['parser', 'ner'])
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [94]:
id2word = corpora.Dictionary(data_lemmatized)
texts = data_lemmatized
corpus = [id2word.doc2bow(text) for text in texts]

Это доказательство того, что mallet не пашет

In [96]:
mallet_path = '/Users/kseniapetuhova/Desktop/mallet-2.0.8' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

CalledProcessError: Command '/Users/kseniapetuhova/Desktop/mallet-2.0.8 import-file --preserve-case --keep-sequence --remove-stopwords --token-regex "\S+" --input /var/folders/dq/3rng4jn93qld39mzf_rljxkc0000gn/T/ded775_corpus.txt --output /var/folders/dq/3rng4jn93qld39mzf_rljxkc0000gn/T/ded775_corpus.mallet' returned non-zero exit status 126.

Напишем функцию, которая будет обучать нашу модель на разном количестве топиков и оценивать значение Coherence Score. Возвращает оптимальное значение параметра num_topics и значение coherence

In [106]:
def find_num_topics(dictionary, corpus, texts, limit, start, step):
    highest_coherence = 0
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
        
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        
        coherence = coherencemodel.get_coherence()
        if coherence > highest_coherence:
            highest_coherence = coherence
            best_number = num_topics
    return best_number, highest_coherence

In [107]:
best_number, highest_coherence = find_num_topics(dictionary=id2word, corpus=corpus,
                                                        texts=data_lemmatized, start=5, limit=31, step=5)

In [108]:
best_number, highest_coherence

(5, 0.556162379991811)

Обучим нашу модель с параметром num_topic=5

In [110]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=best_number, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [111]:
topics = ldamodel.show_topics(formatted=False, num_topics=6)
words_topic = {}
for topic in topics:
    q = topic[1]
    words = {}
    for x in q:
        words[x[0]] = x[1]
    words_topic[topic[0]] = words

Функция, которая находит один главный широкий топик

In [112]:
def find_main_topic(words_topic, text):
    counted_words = {}
    for i in range(len(words_topic)):
        counter = 0
        counted_words[i] = counter
        for word in text:
            if word in words_topic[i].keys():
                counted_words[i] += words_topic[i][word]
    main_topic = collections.Counter(counted_words)
    main_topic = dict(main_topic)
    top_topics = list(main_topic.items())
    top_topics.sort(key=lambda i: i[1], reverse=True)
    topic = top_topics[0][0]
    return topic

In [116]:
texts = data_lemmatized

Создадим список со широкими темами для каждого текста.

In [117]:
all_topics = []
for text in texts:
    topic = find_main_topic(words_topic, text)
    all_topics.append(topic)

In [118]:
bagOfWords = []
for text in texts:
    bag = []
    for word in text:
        bag.append(word)
    bagOfWords.append(bag)

In [119]:
bag_words = []
for text in texts:
    for word in text:
        bag_words.append(word)
uniqueWords = set(bag_words)    

In [120]:
numOfWords = []
for bag in bagOfWords:
    num = dict.fromkeys(uniqueWords, 0)
    for word in bag:
        num[word] += 1
    numOfWords.append(num)

In [121]:
def computeTF(wordDict, bagOfWords):
    tfDict = {}
    bagOfWordsCount = len(bagOfWords)
    for word, count in wordDict.items():
        if count != 0:
            tfDict[word] = count / float(bagOfWordsCount)
    return tfDict

Функция, которая считает идф

In [122]:
import math
def computeIDF(documents):
    N = len(documents)
    
    idfDict = dict.fromkeys(documents[0].keys(), 0)
    for document in documents: # 
        for word, val in document.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        idfDict[word] = math.log(N / float(val))
    return idfDict

In [123]:
idfs = computeIDF(numOfWords)

Функция, которая считает тф_идф

In [124]:
def computeTFIDF(tfBagOfWords, idfs):

    tfidf = {}
    for word, val in tfBagOfWords.items():
        tfidf[word] = val * idfs[word]
    return tfidf

Постчитаем тф и тфидф для каждого текста и с помощью каунтера вытащим по пять слов с каждым высоким тфидф

In [126]:
best = []
for i in range(len(texts)):
    if bagOfWords[i] != 0:
        tf = computeTF(numOfWords[i], bagOfWords[i])
        tfidf = computeTFIDF(tf, idfs)

        best_tfidf = collections.Counter(tfidfs[i]).most_common(5)
        best_words = [word[0] for word in best_tfidf]
        best.append(best_words)

Создадим пандас таблицу с текстом, его широким топиком по дженсиму и 5 тф_идф словами для этого текста

In [127]:
df = pd.DataFrame()
df['text'] = data
df['wide_topic_gensim'] = all_topics
df['5_top_words'] = best

In [128]:
df

,text,wide_topic_gensim,5_top_words
0,From: (wheres my thing) Subject: WHAT car is t...,3,"[car, lerxst, door, funky, where]"
1,From: (Guy Kuo) Subject: SI Clock Poll - Final...,3,"[clock, si, poll, rated, upgrade]"
2,From: (Thomas E Willis) Subject: PB questions....,3,"[display, breifly, machine, hellcat, macleak]"
3,From: (Joe Green) Subject: Re: Weitek P9000 ? ...,3,"[weitek, quadrilateral, amber, division, chip]"
4,From: (Jonathan McDowell) Subject: Re: Shuttle...,1,"[error, warn, bug, waivere, smithsonian_astrop..."
...,...,...,...
11309,From: (Jim Zisfein) Subject: Re: Migraines and...,1,"[patient, dn, eeg, tumor, aunt]"
11310,From: Subject: Screen Death: Mac Plus/512 Line...,0,"[blank, wire, blanking, jiggling, jolt]"
11311,From: (Will Estes) Subject: Mounting CPU Coole...,3,"[cooler, este, cpu, mount, socket]"
11312,From: (Steven Collins) Subject: Re: Sphere fro...,3,"[point, sphere, farin, hyper, graphics_gem]"
